# New York City Bedbug Data - cleaning and processing

In this notebook, I perform cleaning and tranformations on a dataset containing records of bedbug infestations in New York City between 2018 and 2022. I obtain a processed dataset which I use in another notebook to do some analysis on the data. My purpose in making this notebook is to familiarise myself with basic data cleaning and manipulation using pandas.

The dataset used here was obtained from the **NYC Open Data** system [https://data.cityofnewyork.us/Housing-Development/Bedbug-Reporting/wz6d-d3jb]. The description given there for the dataset is as follows.

> Local Law 69 of 2017 requires that all multiple dwelling property owners must attempt to obtain the bed bug infestation history from the tenant or unit owner, including whether eradication measures were employed for a bedbug infestation. The information reported in this file is self-reported information filed annually by property owners.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# The input CSV file here is as downloaded from the link given above, with a single modification, namely the removal
# of an extraneous space character in the column name 'Re-infested Dwelling Unit Count'
full_data = pd.read_csv('Bedbug_Reporting.csv',dtype={'House Number': str, 'BIN':str,
                                                                         'Latitude':str,'Longitude':str})

#Convert date columns to actual date objects
full_data['Filing Date'] = pd.to_datetime(full_data['Filing Date'])
full_data['Filing Period Start Date'] = pd.to_datetime(full_data['Filing Period Start Date'])
full_data['Filling Period End Date'] = pd.to_datetime(full_data['Filling Period End Date'])

In [3]:
#Pull out the relevant columns and rename them for convenience
col_renames = {
'# of Dwelling Units':'#TOT',
'Infested Dwelling Unit Count':'#INF',
'Eradicated Unit Count':'#ERA',
'Re-infested Dwelling Unit Count':'#REI',
'Filing Date':'FD',
'Filing Period Start Date':'FP-S',
'Filling Period End Date':'FP-E',
'Latitude':'LAT',
'Longitude':'LON',
'BIN':'BIN'
}
data = full_data.rename(columns=col_renames)[col_renames.values()]

#For now we'll keep things simple and drop all rows with missing values
num_records = data.shape[0]
data = data.dropna()
print("records dropped: "+str(num_records - data.shape[0])+" out of "+str(num_records))

records dropped: 581 out of 367070


In [4]:
#Have a quick peek at the data
print(data.dtypes)
data.head(10)

#TOT           float64
#INF           float64
#ERA           float64
#REI           float64
FD      datetime64[ns]
FP-S    datetime64[ns]
FP-E    datetime64[ns]
LAT             object
LON             object
BIN             object
dtype: object


,#TOT,#INF,#ERA,#REI,FD,FP-S,FP-E,LAT,LON,BIN
0,6.0,0.0,0.0,0.0,2021-02-16,2019-11-01,2020-10-31,40.727680,-73.949030,3065755
1,3.0,0.0,0.0,0.0,2022-03-04,2020-11-01,2021-10-31,40.613333,-73.994291,3159693
2,3.0,0.0,0.0,0.0,2023-07-10,2021-11-01,2022-10-31,40.684545,-73.945667,3051701
3,4.0,0.0,0.0,0.0,2021-05-05,2019-11-01,2020-10-31,40.702903,-73.902286,4083378
4,3.0,0.0,0.0,0.0,2023-07-06,2021-11-01,2022-10-31,40.656147,-73.903403,3388522
5,6.0,0.0,0.0,0.0,2019-02-07,2017-11-01,2018-11-30,40.724204,-73.946068,3066934
6,2.0,0.0,0.0,0.0,2019-12-20,2018-11-01,2019-10-31,40.735736,-74.003388,1010901
7,3.0,0.0,0.0,0.0,2023-06-23,2021-11-01,2022-10-31,40.828795,-73.894658,2114247
8,9.0,0.0,0.0,0.0,2023-04-13,2021-11-01,2022-10-31,40.731691,-73.843398,4432143
9,8.0,0.0,0.0,0.0,2022-03-18,2020-11-01,2021-10-31,40.828922,-73.924092,2002924


In [5]:
#Do all of the filing periods start in one year and end in the next as we'd expect?
data.loc[data['FP-E'].dt.year != data['FP-S'].dt.year+1]

,#TOT,#INF,#ERA,#REI,FD,FP-S,FP-E,LAT,LON,BIN
112837,3.0,0.0,3.0,0.0,2022-12-01,2021-11-01,2021-10-31,40.890117,-73.837857,2093951
309584,4.0,0.0,0.0,0.0,2022-12-01,2021-11-01,2021-10-31,40.696098,-73.904947,4085605
358097,3.0,0.0,0.0,0.0,2022-12-01,2021-11-01,2021-10-31,40.746426,-73.854634,4049490


In [6]:
#These three rows clearly have a mistake, as the end of the filing period is before the start. It looks like either
#the FP-S year or the FP-E year is off by 1. From the filing date I'd guess that the FP-E should be 2022, but can
#we discount the possibility that the FP-S should be 2020, the FP-E is correct as 2021, and the filing was made
#over a year after the end of the filing period? Do such late filings occur in other records?

bad_ind = (data.loc[data['FP-E'].dt.year != data['FP-S'].dt.year+1]).index
df = data.drop(bad_ind)
max_diff = (df.loc[:,'FD']-df.loc[:,'FP-E']).max()
df = df.loc[(df.loc[:,'FD']-df.loc[:,'FP-E'])==max_diff]
df.shape[0]

87

In [7]:
#So such a long gap is rare but does occur. Let's just drop the problem rows.
data = data.drop(bad_ind)

In [8]:
#Now let's look at the start and end of the reporting period
print('Filing period start dates')
print(data['FP-S'].value_counts().sort_index().to_string())
print('\nFiling period end dates')
print(data['FP-E'].value_counts().sort_index().to_string())

Filing period start dates
2017-11-01     37064
2018-11-01     33758
2019-11-01     96341
2020-11-01     97728
2021-11-01    101595

Filing period end dates
2018-11-30     37064
2019-10-31     33758
2020-10-31     96341
2021-10-31     97728
2022-10-31    101595


In [9]:
#So for each reporting period, all records have the same start and end date. The only complication is that the
#reporting periods for 2017-2018 and 2018-2019 have an overlap (from 2018-11-01 to 2018-11-30, i.e. November 2018).
#This raises the possibility that bedbug infestations (or eradications, or reinfestations) which occured in
#November 2018 might (?) have been reported twice (in both the 2017-2018 and 2018-2019 filing periods). The only
#way I can think of to deal with this would be to reduce all reported totals in 2017-2018 by 1/13, but this would
#would mean non-integer totals in that year. I will just leave things as they are.
#
#So let's condense all the information in FP-S and FP-E into a single 'Year' column (choosing the year in which
#FP-E falls, as that calendar year has the bigger overlap with the filing period)

data['YEAR'] = data['FP-E'].dt.year
data.drop(['FP-S','FP-E'],axis=1,inplace=True)
data.head()

,#TOT,#INF,#ERA,#REI,FD,LAT,LON,BIN,YEAR
0,6.0,0.0,0.0,0.0,2021-02-16,40.727680,-73.949030,3065755,2020
1,3.0,0.0,0.0,0.0,2022-03-04,40.613333,-73.994291,3159693,2021
2,3.0,0.0,0.0,0.0,2023-07-10,40.684545,-73.945667,3051701,2022
3,4.0,0.0,0.0,0.0,2021-05-05,40.702903,-73.902286,4083378,2020
4,3.0,0.0,0.0,0.0,2023-07-06,40.656147,-73.903403,3388522,2022


In [10]:
#Now we need to deal with the fact that a building can have multiple reports in one filing period. The source page
#for the data states that the report with the latest filing date (FD) is the definitive one. Let's look at buildings
#with multiple reports in a year
df = data[['BIN','YEAR']]
count_series = df.value_counts()
#Let's see what the distribution of counts is
print(count_series.value_counts().sort_index().to_string(header=False))

1      311096
2       23444
3        1883
4         354
5          58
6          34
7          14
8           8
9           8
10          2
11          1
12          2
13          1
14          2
15          1
16          1
21          2
23          1
26          1
30          1
31          1
32          1
34          1
37          1
40          1
89          1
90          1
108         1


In [11]:
#So the great majority of buildings have only one or two reports for each year, but some seem to have an
#implausibly high number of reports in one year. Let's have a closer look.
count_series[count_series > 9]

BIN      YEAR
3000000  2022    108
         2020     90
         2021     89
4000000  2022     40
1000000  2020     37
4000000  2020     34
1000000  2021     32
4000000  2021     31
2000000  2022     30
1000000  2022     26
         2018     23
3000000  2018     21
         2019     21
1000000  2019     16
4446424  2022     15
         2020     14
         2021     14
2000000  2021     13
3326506  2020     12
1086159  2021     12
         2022     11
4000000  2018     10
1028563  2020     10
dtype: int64

In [12]:
# It looks like there is something odd going on with the "round million" BINS, i.e. 1000000, 2000000, etc.
# Some digging leads to https://nycplanning.github.io/Geosupport-UPG/chapters/chapterVI/section03/
# where it is explained that these BINs are inserted to fill in missing data, and represent a whole city borough.
# As these records are different in nature, let's split them out into a separate data frame
borough_records = data[data['BIN'].str.contains('000000')]
building_records = data[~data['BIN'].str.contains('000000')]
borough_records.shape[0]

663

In [13]:
#As the number of borough records is so small compared to the building records, we'll ignore them

In [14]:
#Now we want to filter down the building records to have only one record per building per year. Let's calculate the
#number of records we want to end up with
bin_and_year = building_records.loc[:,['BIN','YEAR']]
bin_and_year = bin_and_year.drop_duplicates()
target_record_num = bin_and_year.shape[0]
print(target_record_num)

336898


In [15]:
#For each building and year, choose only those filings with the latest date
idx = building_records.groupby(['BIN', 'YEAR'])['FD'].transform(max) == building_records['FD']
building_records = building_records[idx]

#Let's see if we've got it down to one record per building per year
bin_and_year = building_records.loc[:,['BIN','YEAR']]
bin_and_year = bin_and_year.drop_duplicates()
print(building_records.shape[0] > target_record_num)

True


In [16]:
#So there are still multiple records for some buildings for the same year. Let's take a look
bin_and_year = building_records.loc[:,['BIN','YEAR']]
vcs = bin_and_year.value_counts()
vcs = vcs.sort_values(ascending=False)
vcs.name = 'count'
vcs=vcs.reset_index()

#Let's look at one case
bin0 = vcs.iloc[0]['BIN']
year0 = vcs.iloc[0]['YEAR']
building_records[(building_records['BIN']==bin0) & (building_records['YEAR']==year0)]

,#TOT,#INF,#ERA,#REI,FD,LAT,LON,BIN,YEAR
135346,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018
135347,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018
135348,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018
135349,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018
135350,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018
135351,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018
135352,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018
135353,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018
135354,64.0,0.0,0.0,0.0,2019-01-31,40.652377,-73.973339,3122873,2018


In [17]:
#This seems strange, but we need to deal with it. I don't see any meaningful way of distinguishing between these
#conflicting records, so let's just pick the first
building_records = building_records.groupby(['BIN', 'YEAR']).head(n=1)
print(building_records.shape[0] == target_record_num)

#We can drop the filing dates now, we won't need them again
building_records = building_records.drop('FD',axis=1)

True


In [18]:
print(f"{building_records.shape[0]} records")
br = building_records

#for each BIN find the maximum, minimum, and median number of dwelling units reported in different years
tot_max = br.groupby(['BIN'])['#TOT'].max()
tot_min = br.groupby(['BIN'])['#TOT'].min()
tot_med = br.groupby(['BIN'])['#TOT'].median()
tot_med = tot_med.round()

#find the BINs where there is a difference of more than 20 between the max and min numberof dwelling units reported
#these are the records that we are going to regard as suspicious
tot_diff = tot_max - tot_min
sig_changed_tot_ind = tot_diff[tot_diff > 20].index
print('num rows with significant changes to #TOT: '+str(len(sig_changed_tot_ind)))

#for the next few things, we'll need to index on BIN
br = br.set_index('BIN')

#find those BINs where the reported #TOT values don't seem to be consistent with each other, and drop them
br2 = br.loc[sig_changed_tot_ind]
br2 = br2.loc[:,['#TOT']]
bad_bins = []
for group_name,group_frame in br2.groupby('BIN'):
    if group_frame.shape[0]==1:
        continue
    ser = group_frame.value_counts()
    ind_val_1 = ser[ser==1].index
    ser = ser.drop(ind_val_1)
    if ser.shape[0] != 1:
        bad_bins.append(group_name)      
br = br.drop(bad_bins)

#for the BINs identified above, set all the records to the median
tot_med_sig_ch = tot_med.loc[sig_changed_tot_ind]
br.update(tot_med_sig_ch)

#if any of the above BINs had only 2 records, then drop them, as taking the median here is not a good way to
#resolve the discrepency, and indeed I cannot think of a good way to do this
nums_per_bin = br.groupby(['BIN']).size()
two_ents_ind = nums_per_bin[nums_per_bin == 2].index
to_drop = two_ents_ind.intersection(sig_changed_tot_ind)
br = br.drop(to_drop)

#we don't need to index on BIN anymore
br = br.reset_index()

#Check that the above did what was intended
tot_max = br.groupby(['BIN'])['#TOT'].max()
tot_min = br.groupby(['BIN'])['#TOT'].min()
tot_diff = tot_max - tot_min
sig_changed_tot = tot_diff[tot_diff > 20]
print(f"Sanity check: {sig_changed_tot.shape[0]==0}")

building_records = br
print(f"{building_records.shape[0]} records")

336898 records
num rows with significant changes to #TOT: 1948
Sanity check: True
334775 records


In [19]:
#We'll pause to pick out one BIN and show how the above "set #TOT to the median" operation has cleaned the data

#the data before cleaning
print(data[data['BIN'] == '2093696'])

#the data after cleaning
print(building_records[building_records['BIN'] == '2093696'])

         #TOT  #INF  #ERA  #REI         FD        LAT         LON      BIN  \
6560    100.0   2.0   2.0   0.0 2021-03-31  40.891630  -73.856794  2093696   
16345  2001.0   1.0   1.0   0.0 2022-03-09  40.891630  -73.856794  2093696   
37550   200.0   0.0   0.0   0.0 2019-11-22  40.891630  -73.856794  2093696   
38813   200.0   0.0   0.0   0.0 2019-12-11  40.891630  -73.856794  2093696   

       YEAR  
6560   2020  
16345  2021  
37550  2018  
38813  2019  
           BIN   #TOT  #INF  #ERA  #REI        LAT         LON  YEAR
6005   2093696  200.0   2.0   2.0   0.0  40.891630  -73.856794  2020
14947  2093696  200.0   1.0   1.0   0.0  40.891630  -73.856794  2021
34278  2093696  200.0   0.0   0.0   0.0  40.891630  -73.856794  2018
35424  2093696  200.0   0.0   0.0   0.0  40.891630  -73.856794  2019


In [20]:
#Based on my understanding of the meaning of #TOT,#INF,#ERA,#REI, we should have for all rows that
# #TOT >= #INF >= #ERA >= #REI
#so let's drop any rows that don't satisfy this

br = building_records
ind = br[br['#TOT']<br['#INF']].index
ind = ind.union(br[br['#INF']<br['#ERA']].index)
ind = ind.union(br[br['#ERA']<br['#REI']].index)

print(f"{ind.shape[0]} rows dropped")
building_records = building_records.drop(ind)

4293 rows dropped


In [21]:
#We've now cleaned the data and got it in the form we want
building_records = building_records.reset_index(drop=True) #tidy the index
building_records

,BIN,#TOT,#INF,#ERA,#REI,LAT,LON,YEAR
0,3065755,6.0,0.0,0.0,0.0,40.727680,-73.949030,2020
1,3159693,3.0,0.0,0.0,0.0,40.613333,-73.994291,2021
2,3051701,3.0,0.0,0.0,0.0,40.684545,-73.945667,2022
3,4083378,4.0,0.0,0.0,0.0,40.702903,-73.902286,2020
4,3388522,3.0,0.0,0.0,0.0,40.656147,-73.903403,2022
...,...,...,...,...,...,...,...,...
330477,4450401,8.0,0.0,0.0,0.0,40.764782,-73.772576,2021
330478,4008591,8.0,0.0,0.0,0.0,40.766422,-73.920391,2020
330479,4134129,5.0,0.0,0.0,0.0,40.769538,-73.789856,2021
330480,4451495,528.0,0.0,0.0,0.0,40.727588,-73.816146,2022


In [22]:
#And now let's save it
building_records.to_csv('NYC Bedbugs processed',index=False)